In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from scipy import stats
from reportlab.lib.pagesizes import landscape, A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, Image

# Para qual cenário inflacionário você deseja prever:
inflacao_nova = float(input('Para qual cenário inflacionário você deseja prever: '))
# Para qual volume de vendas você deseja prever a inadimplência:
volume_vendas_novo = float(input('Para qual volume de vendas você deseja prever a inadimplência: '))
# Qual a taxa de desemprego no cenário desejado:
taxa_desemprego_nova = float(input('Qual a taxa de desemprego no cenário desejado: '))
# Qual o endividamento familiar:
endividamento_familiar_nova = float(input('Qual o endividamento familiar no cenário desejado: '))
# Qual o incremento em %:
incremento = float(input('Qual o incremento em %: '))

# Importar dados do arquivo Excel para um DataFrame
dados = pd.read_excel('inadimplencia.xlsx')

# Extrair as colunas de interesse
volume_vendas = dados['volume_vendas']
inflacao = dados['inflacao']
taxa_desemprego = dados['taxa_desemprego']
endividamento_familiar = dados['endividamento_familiar']
previsao_inadimplencia = dados['previsao_inadimplencia']

# Concatenando as variáveis independentes
variaveis_independentes = np.column_stack((inflacao, volume_vendas, taxa_desemprego, endividamento_familiar))

# Criando o objeto StandardScaler
scaler = StandardScaler(copy=False)
variaveis_independentes_scaled = scaler.fit_transform(variaveis_independentes)

# Realizando a transformação dos dados
dados_novos = np.array([[inflacao_nova, volume_vendas_novo, taxa_desemprego_nova, endividamento_familiar_nova],
                       [inflacao_nova * (1 + incremento/100), volume_vendas_novo * (1 + incremento/100),
                        taxa_desemprego_nova * (1 + incremento/100), endividamento_familiar_nova * (1 + incremento/100)]])

dados_novos_scaled = scaler.transform(dados_novos)

# Configurando o modelo de Random Forest
modelo_randomforest = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinando o modelo de Random Forest
modelo_randomforest.fit(variaveis_independentes_scaled, previsao_inadimplencia)

# Fazendo a previsão com o modelo reajustado
previsao_randomforest = modelo_randomforest.predict(dados_novos_scaled)

# Calculando o intervalo de confiança com 98%
residuals = previsao_inadimplencia[:2] - previsao_randomforest
s_err = np.sqrt(np.sum(residuals**2) / (variaveis_independentes_scaled.shape[0] - variaveis_independentes_scaled.shape[1] - 1))
conf = stats.t.ppf(0.99, variaveis_independentes_scaled.shape[0] - variaveis_independentes_scaled.shape[1]) * s_err

inferior, superior = previsao_randomforest - conf, previsao_randomforest + conf

# Configuração do PDF com orientação paisagem e tamanho A4
pdf_filename = "previsao_inadimplencia.pdf"
pdf = SimpleDocTemplate(pdf_filename, pagesize=landscape(A4))
elements = []

# Adicionando a logomarca
logo_filename = "imagem1.png"
logo = Image(logo_filename, width=150, height=100)
elements.append(logo)

# Criação da tabela de previsão
table_data = [
    ["Inflação (%)", "Volume de Vendas", "Taxa de Desemprego (%)", "Endividamento Familiar (%)",
     "Previsão de Inadimplência (%)"],
    [f"{inflacao_nova:.2f}%", "{:.2f}".format(volume_vendas_novo), f"{taxa_desemprego_nova:.2f}%",
     f"{endividamento_familiar_nova:.2f}%", ""],
    [f"{inflacao_nova * (1 + incremento/100):.2f}%", "{:.2f}".format(volume_vendas_novo * (1 + incremento/100)),
     f"{taxa_desemprego_nova * (1 + incremento/100):.2f}%", f"{endividamento_familiar_nova * (1 + incremento/100):.2f}%", ""],
    ["Previsão de Inadimplência com Random Forest:", "", "", "", f"{previsao_randomforest[0]:.2%}"],
    ["Intervalo de Confiança:", "", "", "", f"[{inferior[0]:.2%}, {superior[0]:.2%}]"],
    ["Previsão de Inadimplência com Random Forest (Incremento):", "", "", "", f"{previsao_randomforest[1]:.2%}"],
    ["Intervalo de Confiança (Incremento):", "", "", "", f"[{inferior[1]:.2%}, {superior[1]:.2%}]"]
]

table_style = [
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 8),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
]

table = Table(table_data, style=table_style)

# Estilo do parágrafo
styles = getSampleStyleSheet()
paragrafo_style = ParagraphStyle('paragrafo_style', parent=styles['Normal'])
paragrafo_style.alignment = 1

# Criação do parágrafo de título
title_paragraph = Paragraph('<u>Previsão de Inadimplência</u>', styles['Heading3'])
title_paragraph.alignment = 1  # Centralizar o título

# Adicionando os elementos ao documento PDF
elements.append(title_paragraph)
elements.append(table)
elements.append(Paragraph(f"Fontes: ", styles['Normal']))
elements.append(Paragraph(f"1. Dados internos da empresa", styles['Normal']))
elements.append(Paragraph(f"2. Pesquisas do Instituto Nacional de Estatística", styles['Normal']))
elements.append(Paragraph(f"3. Relatórios do Ministério do Trabalho e Emprego", styles['Normal']))
elements.append(Paragraph(f"4. Índice de Endividamento Familiar do Banco Central", styles['Normal']))
elements.append(Paragraph(f"5. Incremento : {incremento:.2f}%", styles['Normal']))

# Gerando o PDF final
pdf.build(elements)

print(f"A previsão de inadimplência para um cenário com os seguintes parâmetros:")
print(f"Inflação: {inflacao_nova:.2%}")
print(f"Volume de Vendas: {volume_vendas_novo:.2f}")
print(f"Taxa de Desemprego: {taxa_desemprego_nova:.2%}")
print(f"Endividamento Familiar: {endividamento_familiar_nova:.2%}")
print(f"Incremento: {incremento:.2f}%")
print(f"é de: {previsao_randomforest[0]:.2%}")
print(f"O intervalo de confiança é: [{inferior[0]:.2%}, {superior[0]:.2%}]")

print(f"\nA previsão de inadimplência para um cenário com os seguintes parâmetros (com incremento):")
print(f"Inflação: {inflacao_nova * (1 + incremento/100):.2%}")
print(f"Volume de Vendas: {volume_vendas_novo * (1 + incremento/100):.2f}")
print(f"Taxa de Desemprego: {taxa_desemprego_nova * (1 + incremento/100):.2%}")
print(f"Endividamento Familiar: {endividamento_familiar_nova * (1 + incremento/100):.2%}")
print(f"Incremento: {incremento:.2f}%")
print(f"é de: {previsao_randomforest[1]:.2%}")
print(f"O intervalo de confiança é: [{inferior[1]:.2%}, {superior[1]:.2%}]")

print("\nO relatório em PDF foi gerado com sucesso.")
print("Fim do programa.")



Para qual cenário inflacionário você deseja prever:  5
Para qual volume de vendas você deseja prever a inadimplência:  10
Qual a taxa de desemprego no cenário desejado:  30
Qual o endividamento familiar no cenário desejado:  20
Qual o incremento em %:  5


A previsão de inadimplência para um cenário com os seguintes parâmetros:
Inflação: 500.00%
Volume de Vendas: 10.00
Taxa de Desemprego: 3000.00%
Endividamento Familiar: 2000.00%
Incremento: 5.00%
é de: 2.96%
O intervalo de confiança é: [1.91%, 4.00%]

A previsão de inadimplência para um cenário com os seguintes parâmetros (com incremento):
Inflação: 525.00%
Volume de Vendas: 10.50
Taxa de Desemprego: 3150.00%
Endividamento Familiar: 2100.00%
Incremento: 5.00%
é de: 2.96%
O intervalo de confiança é: [1.91%, 4.00%]

O relatório em PDF foi gerado com sucesso.
Fim do programa.


In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from scipy import stats
from reportlab.lib.pagesizes import landscape, A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Table, Image
from tabulate import tabulate
import win32com.client as win32

# Para qual cenário inflacionário você deseja prever:
inflacao_nova = float(input('Para qual cenário inflacionário você deseja prever: '))
# Para qual volume de vendas você deseja prever a inadimplência:
volume_vendas_novo = float(input('Para qual volume de vendas você deseja prever a inadimplência: '))
# Qual a taxa de desemprego no cenário desejado:
taxa_desemprego_nova = float(input('Qual a taxa de desemprego no cenário desejado: '))
# Qual o endividamento familiar:
endividamento_familiar_nova = float(input('Qual o endividamento familiar no cenário desejado: '))
# Qual o incremento em %:
incremento = float(input('Qual o incremento em %: '))

# Importar dados do arquivo Excel para um DataFrame
dados = pd.read_excel('inadimplencia.xlsx')

# Extrair as colunas de interesse
volume_vendas = dados['volume_vendas']
inflacao = dados['inflacao']
taxa_desemprego = dados['taxa_desemprego']
endividamento_familiar = dados['endividamento_familiar']
previsao_inadimplencia = dados['previsao_inadimplencia']

# Concatenando as variáveis independentes
variaveis_independentes = np.column_stack((inflacao, volume_vendas, taxa_desemprego, endividamento_familiar))

# Criando o objeto StandardScaler
scaler = StandardScaler(copy=False)
variaveis_independentes_scaled = scaler.fit_transform(variaveis_independentes)

# Realizando a transformação dos dados
dados_novos = np.array([[inflacao_nova, volume_vendas_novo, taxa_desemprego_nova, endividamento_familiar_nova],
                       [inflacao_nova * (1 + incremento/100), volume_vendas_novo * (1 + incremento/100),
                        taxa_desemprego_nova * (1 + incremento/100), endividamento_familiar_nova * (1 + incremento/100)]])

dados_novos_scaled = scaler.transform(dados_novos)

# Configurando o modelo de Random Forest
modelo_randomforest = RandomForestRegressor(n_estimators=100, random_state=42)

# Treinando o modelo de Random Forest
modelo_randomforest.fit(variaveis_independentes_scaled, previsao_inadimplencia)

# Fazendo a previsão com o modelo reajustado
previsao_randomforest = modelo_randomforest.predict(dados_novos_scaled)

# Calculando o intervalo de confiança com 98%
residuals = previsao_inadimplencia[:2] - previsao_randomforest
s_err = np.sqrt(np.sum(residuals**2) / (variaveis_independentes_scaled.shape[0] - variaveis_independentes_scaled.shape[1] - 1))
conf = stats.t.ppf(0.99, variaveis_independentes_scaled.shape[0] - variaveis_independentes_scaled.shape[1]) * s_err

inferior, superior = previsao_randomforest - conf, previsao_randomforest + conf

# Configuração do PDF com orientação paisagem e tamanho A4
pdf_filename = "previsao_inadimplencia.pdf"
pdf = SimpleDocTemplate(pdf_filename, pagesize=landscape(A4))
elements = []

# Adicionando a logomarca
logo_filename = "imagem1.png"
logo = Image(logo_filename, width=150, height=100)
elements.append(logo)

# Criação da tabela de previsão
table_data = [
    ["Inflação (%)", "Volume de Vendas", "Taxa de Desemprego (%)", "Endividamento Familiar (%)",
     "Previsão de Inadimplência (%)"],
    [f"{inflacao_nova:.2f}%", "{:.2f}".format(volume_vendas_novo), f"{taxa_desemprego_nova:.2f}%",
     f"{endividamento_familiar_nova:.2f}%", ""],
    [f"{inflacao_nova * (1 + incremento/100):.2f}%", "{:.2f}".format(volume_vendas_novo * (1 + incremento/100)),
     f"{taxa_desemprego_nova * (1 + incremento/100):.2f}%", f"{endividamento_familiar_nova * (1 + incremento/100):.2f}%", ""],
    ["Previsão de Inadimplência com Random Forest:", "", "", "", f"{previsao_randomforest[0]:.2%}"],
    ["Intervalo de Confiança:", "", "", "", f"[{inferior[0]:.2%}, {superior[0]:.2%}]"],
    ["Previsão de Inadimplência com Random Forest (Incremento):", "", "", "", f"{previsao_randomforest[1]:.2%}"],
    ["Intervalo de Confiança (Incremento):", "", "", "", f"[{inferior[1]:.2%}, {superior[1]:.2%}]"]
]

table_style = [
    ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 8),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
    ('GRID', (0, 0), (-1, -1), 1, colors.black),
]

table = Table(table_data, style=table_style)

# Estilo do parágrafo
styles = getSampleStyleSheet()
paragrafo_style = ParagraphStyle('paragrafo_style', parent=styles['Normal'])
paragrafo_style.alignment = 1

# Criação do parágrafo de título
title_paragraph = Paragraph('<u>Previsão de Inadimplência</u>', styles['Heading3'])
title_paragraph.alignment = 1  # Centralizar o título

# Adicionando os elementos ao documento PDF
elements.append(title_paragraph)
elements.append(table)
elements.append(Paragraph(f"Fontes: ", styles['Normal']))
elements.append(Paragraph(f"1. Dados internos da empresa", styles['Normal']))
elements.append(Paragraph(f"2. Pesquisas do Instituto Nacional de Estatística", styles['Normal']))
elements.append(Paragraph(f"3. Relatórios do Ministério do Trabalho e Emprego", styles['Normal']))
elements.append(Paragraph(f"4. Índice de Endividamento Familiar do Banco Central", styles['Normal']))
elements.append(Paragraph(f"5. Incremento : {incremento:.2f}%", styles['Normal']))

# Gerando o PDF final
pdf.build(elements)

print(f"A previsão de inadimplência para um cenário com os seguintes parâmetros:")
print(f"Inflação: {inflacao_nova:.2%}")
print(f"Volume de Vendas: {volume_vendas_novo:.2f}")
print(f"Taxa de Desemprego: {taxa_desemprego_nova:.2%}")
print(f"Endividamento Familiar: {endividamento_familiar_nova:.2%}")
print(f"Incremento: {incremento:.2f}%")
print(f"é de: {previsao_randomforest[0]:.2%}")
print(f"O intervalo de confiança é: [{inferior[0]:.2%}, {superior[0]:.2%}]")

print(f"\nA previsão de inadimplência para um cenário com os seguintes parâmetros (com incremento):")
print(f"Inflação: {inflacao_nova * (1 + incremento/100):.2%}")
print(f"Volume de Vendas: {volume_vendas_novo * (1 + incremento/100):.2f}")
print(f"Taxa de Desemprego: {taxa_desemprego_nova * (1 + incremento/100):.2%}")
print(f"Endividamento Familiar: {endividamento_familiar_nova * (1 + incremento/100):.2%}")
print(f"Incremento: {incremento:.2f}%")
print(f"é de: {previsao_randomforest[1]:.2%}")
print(f"O intervalo de confiança é: [{inferior[1]:.2%}, {superior[1]:.2%}]")

print("\nO relatório em PDF foi gerado com sucesso.")
print("Fim do programa.")

table_html = tabulate(table_data, tablefmt="html")

outlook = win32.Dispatch("Outlook.Application")
mail = outlook.CreateItem(0)
mail.Subject = "Relatório de Previsão de Inadimplência"

# Define o corpo do e-mail com o HTML da tabela
mail.HTMLBody = f"<html><body>{table_html}</body></html>"

# Define o corpo do e-mail com o HTML da tabela e estilos inline
mail.HTMLBody = f"""
<html>
<body>
    <h2>Previsão de Inadimplência</h2>
    <table style="width:100%; border: 1px solid #000; border-collapse: collapse;">
        <tr>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Inflação (%)</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Volume de Vendas</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Taxa de Desemprego (%)</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Endividamento Familiar (%)</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Previsão de Inadimplência (%)</th>
        </tr>
        <tr>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{inflacao_nova:.2f}%</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{volume_vendas_novo:.2f}</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{taxa_desemprego_nova:.2f}%</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{endividamento_familiar_nova:.2f}%</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{previsao_randomforest[0]:.2%}</td>
        </tr>
    </table>
    <p>Intervalo de Confiança: [{inferior[0]:.2%}, {superior[0]:.2%}]</p>

    <h2>Previsão de Inadimplência com Incremento</h2>
    <table style="width:100%; border: 1px solid #000; border-collapse: collapse;">
        <tr>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Inflação (%)</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Volume de Vendas</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Taxa de Desemprego (%)</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Endividamento Familiar (%)</th>
            <th style="border: 1px solid #000; padding: 8px; text-align: left;">Previsão de Inadimplência (%)</th>
        </tr>
        <tr>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{inflacao_nova * (1 + incremento/100):.2f}%</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{volume_vendas_novo * (1 + incremento/100):.2f}</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{taxa_desemprego_nova * (1 + incremento/100):.2f}%</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{endividamento_familiar_nova * (1 + incremento/100):.2f}%</td>
            <td style="border: 1px solid #000; padding: 8px; text-align: left;">{previsao_randomforest[1]:.2%}</td>
        </tr>
    </table>
    <p>Intervalo de Confiança (Incremento): [{inferior[1]:.2%}, {superior[1]:.2%}]</p>
    <p>Incremento: {incremento:.2f}%</p>

    <h2>Fontes:</h2>
    <p>1. Dados internos da empresa</p>
    <p>2. Pesquisas do Instituto Nacional de Estatística</p>
    <p>3. Relatórios do Ministério do Trabalho e Emprego</p>
    <p>4. Índice de Endividamento Familiar do Banco Central</p>
    <p>5. Incremento : {incremento:.2f}%</p>
</body>
</html>
"""

# Define o destinatário
mail.To = "jose.magalhaes@rodenstock.com.br"

# Envie o e-mail
mail.Send()

print("E-mail enviado com sucesso.")

Para qual cenário inflacionário você deseja prever:  5
Para qual volume de vendas você deseja prever a inadimplência:  5000
Qual a taxa de desemprego no cenário desejado:  11
Qual o endividamento familiar no cenário desejado:  25
Qual o incremento em %:  5


A previsão de inadimplência para um cenário com os seguintes parâmetros:
Inflação: 500.00%
Volume de Vendas: 5000.00
Taxa de Desemprego: 1100.00%
Endividamento Familiar: 2500.00%
Incremento: 5.00%
é de: 1.51%
O intervalo de confiança é: [1.12%, 1.91%]

A previsão de inadimplência para um cenário com os seguintes parâmetros (com incremento):
Inflação: 525.00%
Volume de Vendas: 5250.00
Taxa de Desemprego: 1155.00%
Endividamento Familiar: 2625.00%
Incremento: 5.00%
é de: 1.27%
O intervalo de confiança é: [0.87%, 1.66%]

O relatório em PDF foi gerado com sucesso.
Fim do programa.


com_error: (-2146959355, 'Falha na execução do servidor', None, None)